# IBM Qiskit – Quantum Experiment Runner
Runs the VQC on selected features using the modular code in `src/`.

In [ ]:
import os, sys, yaml
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path: sys.path.insert(0, project_root)
from src.data_loader import load_csvs, merge_on_transaction_id
from src.preprocessing import PreprocessConfig, preprocess_pipeline, split_data
from src.model_quantum import QuantumConfig, train_vqc
from src import evaluation as eval_mod
cfg_path = os.path.join(project_root, 'configs', 'config.yaml')
cfg = yaml.safe_load(open(cfg_path, 'r'))
# Load + preprocess
df_txn, df_id = load_csvs(cfg['data']['transaction_csv'], cfg['data']['identity_csv'])
df = merge_on_transaction_id(df_txn, df_id)
pp_cfg = PreprocessConfig(**{k: cfg['preprocessing'][k] for k in ['missing_threshold','target_col','top_k_corr_features']}, id_cols=cfg['preprocessing'].get('id_cols', []))
df_processed, selected = preprocess_pipeline(df, pp_cfg)
X_train, X_test, y_train, y_test = split_data(df_processed, target=pp_cfg.target_col, test_size=cfg['preprocessing']['test_size'], random_state=cfg['preprocessing']['random_state'], stratify=cfg['preprocessing']['stratify'])
# Train VQC
q_cfg = QuantumConfig(num_features=X_train.shape[1], reps_feature_map=cfg['quantum_model']['reps_feature_map'], reps_ansatz=cfg['quantum_model']['reps_ansatz'], optimizer_maxiter=cfg['quantum_model']['optimizer_maxiter'], shots=cfg['quantum_model']['shots'])
clf_q = train_vqc(X_train.values, y_train.values, q_cfg)
y_pred = clf_q.predict(X_test.values)
try:
    y_proba = clf_q.predict_proba(X_test.values)[:,1]
except Exception:
    y_proba = None
eval_mod.compute_metrics(y_test, y_pred, y_proba)